In [17]:
import asyncio
import aiohttp
import json
from pprint import pprint

from attraction_api import fetch_attractions
from hotel_api import HotelRetriever
from flight_api import FlightRetriever
from llm_retrieve import llm_retrieve

In [18]:
itinerary = llm_retrieve()
print("Loaded cities and parameters:")
for city, info in itinerary.items():
    print(f" • {city}: Arrival={info['Arrival Date']}, Departure={info['Departure Date']}, Airport={info['Airport']}")



Loaded cities and parameters:
 • Leh: Arrival=2025-07-12, Departure=2025-07-14, Airport=IXL
 • Srinagar: Arrival=2025-07-14, Departure=2025-07-16, Airport=SXR
 • Shimla: Arrival=2025-07-16, Departure=2025-07-19, Airport=SLV


In [19]:
async def gather_all():
    async with aiohttp.ClientSession() as session:
        attraction_tasks = [fetch_attractions(session, city) for city in itinerary.keys()]
        attraction_results = await asyncio.gather(*attraction_tasks)

    hotel_results = HotelRetriever(itinerary, numadults=2)
    flight_results = FlightRetriever(itinerary)

    return attraction_results, hotel_results, flight_results


In [20]:
# Cell 4: Run & Merge
attractions, hotels, flights = await gather_all()

# Merge results by city
combined = {city: {"city": city, "attractions": [], "hotels": [], "flights": []} for city in itinerary}

# Add attractions
for entry in attractions:
    combined[entry["city"]]["attractions"] = entry["attractions"]

# Add hotels
for entry in hotels:
    city = entry["city"]
    combined[city]["hotels"] = entry["hotels_by"]

# Add flights
for entry in flights:
    from_city = entry["from"]
    to_city = entry["to"]
    date = entry["date"]
    flights_data = entry.get("flights")
    note = entry.get("note")

    # Associate the flight with departure city
    combined[from_city].setdefault("flights", []).append({
        "to": to_city,
        "date": date,
        "flights": flights_data,
        "note": note
    })
    

[Attraction] 🔍 Fetching location_id for: Leh
[Attraction] 🔍 Fetching location_id for: Srinagar
[Attraction] 🔍 Fetching location_id for: Shimla
[Attraction] ✅ location_id for Shimla = 304552
[Attraction] 🔍 Fetching attractions for Shimla
[Attraction] ✅ location_id for Srinagar = 297623
[Attraction] 🔍 Fetching attractions for Srinagar
[Attraction] ✅ location_id for Leh = 4266551
[Attraction] 🔍 Fetching attractions for Leh
[Attraction] 🌐 Raw results count for Srinagar: 20
[Attraction] 🎯 Returning 10 attractions for Srinagar
[Attraction] 🌐 Raw results count for Leh: 20
[Attraction] 🎯 Returning 10 attractions for Leh
[Attraction] 🌐 Raw results count for Shimla: 20
[Attraction] 🎯 Returning 10 attractions for Shimla


In [21]:
final_list = list(combined.values())

print("\n\u2705 After merging, sample output for first city:")
print(json.dumps(final_list[0], indent=2))


✅ After merging, sample output for first city:
{
  "city": "Leh",
  "attractions": [
    {
      "name": "Coca-Cola Park",
      "num_reviews": "647",
      "rating": "4.7",
      "ranking_position": "1",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "56",
          "name": "Fun & Games"
        }
      ]
    },
    {
      "name": "SteelStacks",
      "num_reviews": "661",
      "rating": "4.6",
      "ranking_position": "3",
      "neighborhood_info": null,
      "description": "SteelStacks is a ten-acre campus in Bethlehem, PA, dedicated to arts, culture, family events, community celebrations, education and fun. The campus is in the shadow of the blast furnaces of the former Bethlehem Steel plant.",
      "subcategory": [
        {
          "key": "26",
          "name": "Shopping"
        },
        {
          "key": "49",
          "name": "Museums"
        }
      ]
    },
    {
      "name": "Allentown Fish Hatcher

In [22]:
import pandas as pd
from IPython.display import display

# Flatten into a DataFrame (one row per city)
df = pd.json_normalize(final_list, sep="_")
display(df)

# Save JSON
out_path = "combined_data_hotel_attraction_flight_final.json"
with open(out_path, "w") as f:
    json.dump(final_list, f, indent=2)

print("\u2705 Saved combined results to", out_path)

,city,attractions,flights,hotels_bayesian_review_score,hotels_popularity,hotels_price
0,Leh,"[{'name': 'Coca-Cola Park', 'num_reviews': '64...","[{'to': 'Srinagar', 'date': '2025-07-14', 'fli...","[{'name': 'Tendance et Charmant F2', 'price': ...","[{'name': 'All Suites Appart Hotel Le Havre', ...",[{'name': 'Bel appartement avec parking et ter...
1,Srinagar,"[{'name': 'Dal Lake', 'num_reviews': '4329', '...","[{'to': 'Shimla', 'date': '2025-07-16', 'fligh...","[{'name': 'John's Homestay', 'price': 55.1}, {...","[{'name': 'Houseboat Zaindari Palace', 'price'...","[{'name': 'LalaApartment', 'price': 13.07}, {'..."
2,Shimla,"[{'name': 'Jakhu Hanuman Temple', 'num_reviews...",[],"[{'name': 'Wildflower Hall, An Oberoi Resort, ...",[{'name': 'Victory Hotel and Restaurant by GS ...,"[{'name': 'Staynest Mashobra with balcony', 'p..."


✅ Saved combined results to combined_data_hotel_attraction_flight_final.json


In [23]:
destination = "India"
from_date = "2025-07-12"
to_date = "2025-07-19"
traveller_count = 4
budget = 2000

structured_output = {
    "user_info": {
        "destination": destination,
        "from_date": from_date,
        "to_date": to_date,
        "budget_in_usd": budget,
        "traveller_count": traveller_count
    },
    "data": final_list
}

with open("combined_data_with_user_info.json", "w") as f:
    json.dump(structured_output, f, indent=2)
# print("✅ Saved structured travel data to structured_travel_data.json")


### below code is for testing the final prompt, will be removed from this file and will add it to the travelgenie_rag_ui 

In [24]:
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama

with open("combined_data_with_user_info.json") as f:
    full_data = json.load(f)

user_info = full_data["user_info"]
data_list = full_data["data"]


In [25]:

prompt_template = PromptTemplate(
    input_variables=["destination", "from_date", "to_date", "budget", "travellers", "data"],
    template="""
You are an expert travel planner AI.

Use the structured JSON data provided below to generate a complete day-by-day travel itinerary for the user.
Include:
- Clear day-wise breakdown (Day 1, Day 2...)
- Top attractions in each city
- Flights with departure/arrival details
- Recommended hotels with pricing
- Restaurant suggestions per city (search good ones or suggest based on popular areas)
- End summary with cost breakdown and travel tips.

User Info:
Destination: {destination}
Dates: {from_date} to {to_date}
Budget: ${budget}
Travellers: {travellers}

Structured Trip Data:
{data}

Now write a friendly, professional and complete travel itinerary in natural language.
"""
)

def get_llm():
    return ChatOllama(model="llama3.2")


In [26]:
def generate_itinerary():
    prompt = prompt_template.format(
        destination=user_info["destination"],
        from_date=user_info["from_date"],
        to_date=user_info["to_date"],
        budget=user_info["budget_in_usd"],
        travellers=user_info["traveller_count"],
        data=json.dumps(data_list, indent=2)
    )

    llm = get_llm()
    result = llm.invoke(prompt)
    return result.content if hasattr(result, 'content') else result

narrative = generate_itinerary()

with open("generated_itinerary.txt", "w") as f:
    f.write(narrative)